In [72]:
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.spatial import Delaunay
import math

class setup():
    def __init__(self):
        # global lithology
        global color_list, color_count, color
        global file_count
        global elev, azim, grid
        color = ['coral', 'coral']
        color_count = 0
        file_count = 0
        elev = 40
        azim = 150
        grid = 1
        # Deleted content: plt.closefig
        
class deposition:
   # Code to generate arrays for defined accuracy
    inten = 31
    x1 = []
    y1 = []
    x1_temp = list(np.linspace(0, 10, inten))
    for i in range(inten):
        x1 = x1 + x1_temp
    for i in range(inten**2):
        y1.append(x1_temp[i//inten])
    z1 = [1 for i in range(inten**2)]
    z12 = [2 for i in range(inten**2)]
    xy110 = x1_temp
    xy1 = [-0.1 for i in range(inten)]
    xy10 = [10.1 for i in range(inten)]
    z2 = [0 for i in range(inten*4)]
    z22 = [1 for i in range(inten*4)]

    x1.extend(xy1)
    x1.extend(xy110)
    x1.extend(xy10)
    x1.extend(xy110)
    y1.extend(xy110)
    y1.extend(xy1)
    y1.extend(xy110)
    y1.extend(xy10)
    z1.extend(z2)
    z12.extend(z22)
    
    surfx = [np.array(x1, dtype=float), np.array(x1, dtype=float)]
    surfy = [np.array(y1, dtype=float), np.array(y1, dtype=float)]
    surfz = [np.array(z1, dtype=float), np.array(z12, dtype=float)]

    basez = []
    topz = []
    topptz = 0
    baseptz = 0
    deposition_array_x = np.array(x1)
    deposition_array_y = np.array(y1)
    
    # New feature -> define unit thickness
    unit_thickness = 0.5

    def __init__(self, thickness):
        global lithology
        global color, color_list, color_count
        for i in range(int(thickness / deposition.unit_thickness)):
            color.append(color_list[color_count])
        color_count += 1

    def reset():
        deposition.surfx = [np.array(deposition.x1, dtype=float)]
        deposition.surfy = [np.array(deposition.y1, dtype=float)]
        deposition.surfz = [np.array(deposition.z1, dtype=float)]
        erosion.eroded = False
        print("Model parameters have been resetted!")
        
    def topsurfz(thickness):
        deposition.topz = []
        for z in deposition.surfz:
            deposition.topz.append(max(z))
        deposition.topptz = max(deposition.topz)
    
    def coordinates(thickness):
        deposition(thickness)
        deposition.topsurfz(thickness)
        for i in range(int(thickness / deposition.unit_thickness)):
            deposition_array_x = np.array(deposition.x1)
            deposition_array_y = np.array(deposition.y1)
            deposition.surfx.append(deposition_array_x)
            deposition.surfy.append(deposition_array_y)   
            if erosion.eroded == True:
                # deposition.topptz -= 1
                deposition.topptz = erosion.uplim - 1
                bottom_array = np.full(deposition.inten*4, deposition.topptz, dtype=float)
                erosion.eroded = False
            else:
                bottom_array = np.full(deposition.inten*4, deposition.topptz, dtype=float)
            deposition.topptz += deposition.unit_thickness
            # level = deposition.topptz + thickness
            level_array = np.full(deposition.inten*deposition.inten, deposition.topptz, dtype=float)
            combined_array = np.concatenate((level_array, bottom_array))
            deposition.surfz.append(combined_array)  

class erosion:
    lowlim = -100
    limit = None
    eroded = False
    def __init__(self):
        erosion.eroded = True
   
    def surfacez(npx, npy, npz, uplim):
        tempx = []
        tempy = []
        tempz = []
        for i in range(len(npx)):
            if npz[i] <= uplim:
                tempx.append(npx[i])
                tempy.append(npy[i])
                tempz.append(npz[i])
        return tempx, tempy, tempz
    
    def remove_bed(surfx, surfy, surfz):
        # Remove the beds that have less than three points uneroded
        for i in reversed(range(len(surfx))):
            if len(surfx[i]) < 3:
                del surfx[i]
                del surfy[i]
                del surfz[i]
                del color[i]
            # Remove beds that have all points on a straight line
            try:
                setx = len(set(surfx[i]))
                sety = len(set(surfy[i]))
                setz = len(set(surfz[i]))
                if setx == 1 or sety == 1 or setz == 1:
                    del surfx[i]
                    del surfy[i]
                    del surfz[i]
                    del color[i]
            except:
                pass
        return surfx, surfy, surfz
        
    
    def coordinates(uplim):
        for i in range(len(deposition.surfx)):
            deposition.surfx[i], deposition.surfy[i], deposition.surfz[i] = erosion.surfacez(deposition.surfx[i], 
                                                                deposition.surfy[i], deposition.surfz[i], uplim)
        deposition.surfx, deposition.surfy, deposition.surfz = erosion.remove_bed(deposition.surfx, deposition.surfy, deposition.surfz)

class uplift:
    
    def deltaz(a, b, ulangle, ulbearing):
        # Eqt 1: y = mx + c
        # Eqt 2: b = (1/-m)a + d
        if ulbearing == 0 or ulbearing == 180: 
            x = a
            y = 5
        elif ulbearing == 90 or ulbearing == 270:
            x = 5
            y = b
        else:
            m = np.tan(np.radians(ulbearing))
            c = 5 - m * 5
            d = b - (1/-m)*a
            # print('m =', m)
            # print('c =', c)
            # print('d =', d)
            # x = (b*m - m*c + a) / (1 + m**2)
            x = (m*d - m*c)/(m**2 + 1) 
            y = m*x + c
            # Compute Eqt 3
            c3 = 5 - (1/-m)*5
        ld = 5/np.cos(np.radians(ulbearing))
        sd = np.sqrt((x - 5)**2 + (y - 5)**2)
        ratio = sd/ld
        # print('ld =', ld)
        # print('sd =', sd)
        # print('ratio =', ratio)
        # z = ratio * ((10/np.cos(np.radians(ulbearing)))/ulangle)
        # dz = ratio * ld/np.cos(np.radians(ulangle))
        dz = sd * np.tan(np.radians(ulangle))
        if ulbearing == 0 or ulbearing == 180: 
            checkeqt = a - 5
            if ulbearing == 0:
                if checkeqt >= 0:
                    dz = -dz
            if ulbearing == 180:
                if checkeqt <= 0:
                    dz = -dz
        elif ulbearing == 90 or ulbearing == 270:
            checkeqt = b - 5
            if ulbearing == 90:
                if checkeqt >= 0:
                    dz = -dz
            if ulbearing == 270:
                if checkeqt <= 0:
                    dz = -dz
        else:
            checkeqt = (1/-m)*a - b + c3
            if ulbearing < 180:
                if checkeqt < 0:
                    dz = -dz
            else:
                if checkeqt > 0:
                    dz = -dz
        # print('z =',z)
        # print('dz =', dz)
        # print()
        return dz
    
    def coordinates(ulangle, ulbearing):
        for i in range(len(deposition.surfx)):
            for n in range(len(deposition.surfx[i])):
                dz = uplift.deltaz(deposition.surfx[i][n], deposition.surfy[i][n], ulangle, ulbearing)
                # diffz = 5 - deposition.surfz[i][n]
                deposition.surfz[i][n] += dz
                # print(deposition.surfz[i][n])
    
class folding:
    def __init__(self):
        # global fdir, flength
        fxp = None     # fold axial plane equation: z = mx + ny + c
        fxpm = 0
        fxpn = 0
        fxpc = 0
        
    def foldz(ix, iy, iz, fdir, cline, flength):
        if cline == 's':
            flength = -flength
        if fdir == 'x':
            dz = (-1/(flength))*(ix - 5)**2 - 3
            # dz = -1/5*(ix - 5)**2
            # dz = 3*np.sin((3/4)*ix - 1)
            # dz = 1/4*(ix - 5)**2 - 2
            # folding.fxp = x - 5
            folding.fxpm = [1]
            folding.fxpn = [0]
            folding.fxpc = [-5]
        elif fdir == 'y':
            dz = (-1/(flength))*(iy - 5)**2 - 3
            # dz = -1/4*(iy - 5)**2 + 3
            # dz = 3*np.sin(np.radians((1/2)*iy - 1))
            # folding.fxp = y - 5
            folding.fxpm = [0]
            folding.fxpn = [1]
            folding.fxpc = [-5]
        elif fdir == 'yas':
            dz = flength*np.sin(iy*(0.85))
            # x*folding.fxpm + y*folding.fxpn + folding.fxpc
            folding.fxpm = [0, 0, 0]
            folding.fxpn = [1, 1, 1]
            folding.fxpc = [-30*(math.pi)/17, -10*(math.pi)/17, -50*(math.pi)/17]
        elif fdir == 'xas':
            dz = flength*np.sin(ix*(0.7))
            folding.fxpm = [0]
            folding.fxpn = [1]
            folding.fxpc = [-5]
            
        z = iz + dz
        return z
    
    def fxp_split(dsx, dsy, dsz, color):
        ndx = []
        ndy = []
        ndz = []
        ncolor = []
        print('len(deposition.surfx) =', len(deposition.surfx))
        print('len(colour) =', len(color_list))
        for plane in range(len(folding.fxpm)):
            print('plane =', plane)
            for i in reversed(range(len(dsx))):
                tempx1 = np.array([], dtype=float)
                tempy1 = np.array([], dtype=float)
                tempz1 = np.array([], dtype=float)
                tempx2 = np.array([], dtype=float)
                tempy2 = np.array([], dtype=float)
                tempz2 = np.array([], dtype=float)
                for (x, y, z) in zip(dsx[i], dsy[i], dsz[i]):
                    check = x*folding.fxpm[plane] + y*folding.fxpn[plane] + folding.fxpc[plane]
                    if check >= 0:
                        # print('x, y, z =', x, y, z)
                        # print('check =', check)
                        # print()
                        tempx1 = np.append(tempx1, x)
                        tempy1 = np.append(tempy1, y)
                        tempz1 = np.append(tempz1, z)
                    if check <= 0.5:
                        tempx2 = np.append(tempx2, x)
                        tempy2 = np.append(tempy2, y)
                        tempz2 = np.append(tempz2, z)
                # print(tempx1)
                if tempx1.size != 0:
                    ndx.append(tempx1)
                    ndy.append(tempy1)
                    ndz.append(tempz1)
                    ncolor.append(color[i])
                if tempx2.size != 0:
                    ndx.append(tempx2)
                    ndy.append(tempy2)
                    ndz.append(tempz2)
                    ncolor.append(color[i])
        return ndx, ndy, ndz, ncolor
                
    def coordinates(fdir, cline, flength):
        global color
        for i in range(len(deposition.surfx)):
            for n in range(len(deposition.surfx[i])):
                z = folding.foldz(deposition.surfx[i][n], deposition.surfy[i][n], deposition.surfz[i][n], fdir, cline, flength)
                deposition.surfz[i][n] = z
        # Split the beds along the fold axial plane
        ndx, ndy, ndz, ncolor = folding.fxp_split(deposition.surfx, deposition.surfy, deposition.surfz, color)
        deposition.surfx, deposition.surfy, deposition.surfz, color = ndx, ndy, ndz, ncolor
        deposition.surfx, deposition.surfy, deposition.surfz = erosion.remove_bed(deposition.surfx, deposition.surfy, deposition.surfz)

class faulting:
    def programing3d(listx, listy, listz, color):
        global strike, dip, disp
        throw = disp*np.sin(np.radians(dip))
        print('strike, dip =', strike, dip)
        fa = 1
        fb = -1/(np.tan(np.radians(strike)))*fa 
        fc = np.sqrt(fa**2 + fb**2)*np.tan(np.radians(dip))
        if strike > 180:
            fc = -fc
            throw = -throw
        fx, fy, fz = 5, 5, 2
        fd = -fa*fx - fb*fy - fc*fz
        dx, dy, dz = fa*(throw/fc), fb*(throw/fc), throw   # dxyz = scaled fxyz
        ex, ey, ez = 5, 5, 2
        bx, by, bz = 5, 4, 2
        gx, gy, gz = 3, 4, 2
        bx = fx + dx
        by = fy + dy
        bz = fz + dz
        ex = fx + 1
        ey = fy + np.tan(np.radians(strike))
        ez = fz
        npe = np.array([ex - fx, ey - fy, ez - fz])
        npb = np.array([bx - fx, by - fy, bz - fz])
        npc = np.cross(npe, npb)
        a, b, c = npc[0], npc[1], npc[2]
        d = -a*fx - b*fy - c*fz
        npx = []
        npy = []
        npz = []
        ncolor = []
        for n in reversed(range(len(listx))):
            # Create arrays to store the new coordinates temporarily
            tempx1 = np.array([], dtype=float)
            tempy1 = np.array([], dtype=float)
            tempz1 = np.array([], dtype=float)
            tempx2 = np.array([], dtype=float)
            tempy2 = np.array([], dtype=float)
            tempz2 = np.array([], dtype=float)
            for i in range(len(listx[n])):
                if i == 525:
                    # print(range(len(listz[n])))
                    pass
                try:
                    check = a*listx[n][i] + b*listy[n][i] + c*listz[n][i] + d
                except:
                    # print(range(len(listz[n])))
                    pass
                if check > 0.5:
                    listx[n][i] += dx
                    listy[n][i] += dy
                    listz[n][i] += dz
                    tempx1 = np.append(tempx1, listx[n][i])
                    tempy1 = np.append(tempy1, listy[n][i])
                    tempz1 = np.append(tempz1, listz[n][i])
                    if n == 3:
                        if i == 5:
                            print('list[n] =', listx[n])
                            print('listx[n - 1](1) =', listx[n - 1])
                elif check < -0.5:
                    tempx2 = np.append(tempx2, listx[n][i])
                    tempy2 = np.append(tempy2, listy[n][i])
                    tempz2 = np.append(tempz2, listz[n][i])
                    if n == 3:
                        if i == 5:
                            print('listx[n - 1](2) =', listx[n - 1])
                else:
                    tempx1 = np.append(tempx1, listx[n][i] + dx)
                    tempy1 = np.append(tempy1, listy[n][i] + dy)
                    tempz1 = np.append(tempz1, listz[n][i] + dz)
                    tempx2 = np.append(tempx2, listx[n][i])
                    tempy2 = np.append(tempy2, listy[n][i])
                    tempz2 = np.append(tempz2, listz[n][i])
                    if n == 3:
                        if i == 5:
                            print('listx[n - 1](3) =', listx[n - 1])
                if n == 3:
                    if i == 5:
                        print('listx[n - 1](4) =', listx[n - 1])
            # Append the new arrays to list
            if tempx1.size != 0:
                ncolor.append(color[n])
                npx.append(tempx1)
                npy.append(tempy1)
                npz.append(tempz1)
            if tempx2.size != 0:
                ncolor.append(color[n])
                npx.append(tempx2)
                npy.append(tempy2)
                npz.append(tempz2)
        return npx, npy, npz, ncolor
    
    def remove_points(surfx, surfy, surfz):
        newpointx = np.array([])
        newpointy = np.array([])
        newpointz = np.array([])
        # Remove the points that are out of bounds
        for i in reversed(range(len(surfx))):
            for n in reversed(range(len(surfx[i]))):
                if (surfx[i][n] > 10.5 or surfx[i][n] < -0.5 or surfy[i][n] > 10.5 or surfy[i][n] < -0.5):
                    surfx[i] = np.delete(surfx[i], n)
                    surfy[i] = np.delete(surfy[i], n)
                    surfz[i] = np.delete(surfz[i], n)
            if i == 0:
                # print(deposition.surfy[i])
                pass
        return surfx, surfy, surfz

    def coordinates():
        global color
        ndx, ndy, ndz, ncolor = faulting.programing3d(deposition.surfx, deposition.surfy, deposition.surfz, color)
        deposition.surfx, deposition.surfy, deposition.surfz, color = ndx, ndy, ndz, ncolor
        deposition.surfx, deposition.surfy, deposition.surfz = faulting.remove_points(deposition.surfx, deposition.surfy, deposition.surfz)
        deposition.surfx, deposition.surfy, deposition.surfz = erosion.remove_bed(deposition.surfx, deposition.surfy, deposition.surfz)

class plot:
    def plot():
        beds = [0 for i in range(len(deposition.surfx))]
        beds_counter = 0
        data = []
        for (x, y, z, c) in zip(deposition.surfx, deposition.surfy, deposition.surfz, color):
            data.append(go.Mesh3d(x=x, y=y, z=z, color=c))
        # beds_list = [beds[i] for i in range(len(deposition.surfx))]
        fig = go.Figure(data=data)

        # Set aspect ratio
        fig.update_layout(
            scene = dict(
                xaxis = dict(nticks=4, range=[-0.5, 10.5],),
                yaxis = dict(nticks=4, range=[-0.5, 10.5],),
                zaxis = dict(nticks=4, range=[-0.5, 10.5],),),
            width=700,
            margin=dict(r=20, l=10, b=10, t=10))
        
        fig.update_layout(scene_aspectmode='cube')

        fig.show()   

## Run this cell to reset model parameter

In [73]:
# Reset
setup()
deposition.reset()

Model parameters have been resetted!


In [74]:
# Recumbent fold
color_list = ['slategrey', 'wheat', 'darkseagreen', 'steelblue', 'red', 'green', 'blue']

# Deposition
deposition_thickness = [3, 2, 2, 2, 1, 2, 3]
for thickness in deposition_thickness:
    deposition.coordinates(thickness)

# Uplift
ulangle, ulbearing = 40, 180
uplift.coordinates(ulangle, ulbearing)

# Folding - F2
fdir, cline, flength = 'x', 'a', 6
folding.coordinates(fdir, cline, flength)

# Erosion
uplim = 4
erosion.coordinates(uplim)

# Plot
plot.plot()

len(deposition.surfx) = 31
len(colour) = 7
plane = 0
